In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_excel('Online Retail.xlsx')

In [6]:
df1 = df

In [7]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Exploring and Cleaning Data

In [8]:
df1.Country.nunique()

38

In [9]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Malta', 'RSA'], dtype=object)

In [10]:
customer_country=df1[['Country', 'CustomerID']].drop_duplicates()

In [13]:
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID',ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [14]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [18]:
#check for missing values in each column
df1.isnull().sum(axis=0)

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [19]:
#removing missing customer IDs
df1 = df1[pd.notnull(df1['CustomerID'])]

In [21]:
#check minimum values of unitprice and quanitity
df1.Quantity.min()

-80995

In [22]:
df.UnitPrice.min()

-11062.059999999999

In [27]:
df1 = df1[(df1['Quantity']>0)]

In [29]:
df1.shape

(354345, 8)

In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [31]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [34]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

## RFM Customer Segmentation

In [35]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [37]:
import datetime as dt
NOW = dt.datetime(2011,12,10)

In [38]:
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [44]:
#Creating an RFM Table
rfmTable = df1.groupby('CustomerID').agg(
    {
        'InvoiceDate': lambda x: (NOW - x.max()).days, 
        'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()
     }
)
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(
    columns={
        'InvoiceDate': 'recency', 
        'InvoiceNo': 'frequency', 
        'TotalPrice': 'monetary_value'}, 
    inplace=True
)

In [45]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


In [48]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [49]:
quantiles

{'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'monetary_value': {0.25: 300.03999999999996,
  0.5: 651.82000000000016,
  0.75: 1575.8900000000003},
 'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0}}

In [50]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [52]:
segmented_rfm = rfmTable

segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


In [60]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)

segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


In [61]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111


## Best Customers: Bought recently and most often, and spend the most

### Marketing: No price incentives, new products, and loyalty programs

In [63]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111


## Loyal Customers: Buy Most Frequently

### Marketing: Use R and M to Further Segment

In [67]:
segmented_rfm[segmented_rfm['f_quartile']==1].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16029.0,38,242,81024.84,2,1,1,211
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111


## Big Spenders

### Marketing: Market most expensive product

In [70]:
segmented_rfm[segmented_rfm['m_quartile']==1].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
16446.0,0,3,168472.50,1,4,1,141
17511.0,2,963,91062.38,1,1,1,111
16029.0,38,242,81024.84,2,1,1,211
12346.0,325,1,77183.60,4,4,1,441
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111


## Almost Lost: havent purchased in some time, but purchased frequently and spend the most

### Marketing: Aggressive price incentives

In [78]:
step1 = segmented_rfm[segmented_rfm['r_quartile']==3].sort_values('monetary_value', ascending=False)
step2 = step1[step1['f_quartile']==1].sort_values('monetary_value', ascending=False)
step3 = step2[step2['m_quartile']==1].sort_values('monetary_value', ascending=False)
step3

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
16180.0,100,162,10254.18,3,1,1,311
14952.0,59,138,8099.49,3,1,1,311
16745.0,86,357,7194.30,3,1,1,311
16652.0,58,118,6773.97,3,1,1,311
17509.0,57,366,6115.14,3,1,1,311
16984.0,88,407,4481.35,3,1,1,311
15874.0,63,120,4405.88,3,1,1,311
13555.0,73,136,4149.28,3,1,1,311
15416.0,64,192,3984.32,3,1,1,311


## Lost Customers: Haven't purhased for some time, but purchased frequently and spend the most

### Marketing: Aggressive price incentives

In [79]:
step1 = segmented_rfm[segmented_rfm['r_quartile']==4].sort_values('monetary_value', ascending=False)
step2 = step1[step1['f_quartile']==1].sort_values('monetary_value', ascending=False)
step3 = step2[step2['m_quartile']==1].sort_values('monetary_value', ascending=False)
step3

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
13093.0,275,159,7832.470,4,1,1,411
17850.0,372,297,5391.210,4,1,1,411
15808.0,306,208,3734.970,4,1,1,411
15379.0,169,194,3703.290,4,1,1,411
13952.0,217,137,3251.071,4,1,1,411
17504.0,206,127,2997.030,4,1,1,411
12840.0,143,113,2726.770,4,1,1,411
18260.0,172,134,2643.200,4,1,1,411
16919.0,156,327,2596.450,4,1,1,411


## Lost Cheap Customers: Last purchased long ago, purchased few, and spent little

### Marketing: Don't spend too much to reacquire

In [80]:
step1 = segmented_rfm[segmented_rfm['r_quartile']==4].sort_values('monetary_value', ascending=False)
step2 = step1[step1['f_quartile']==4].sort_values('monetary_value', ascending=False)
step3 = step2[step2['m_quartile']==4].sort_values('monetary_value', ascending=False)
step3

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
15609.0,184,16,298.48,4,4,4,444
17678.0,263,16,298.11,4,4,4,444
12881.0,275,7,298.00,4,4,4,444
17531.0,190,14,296.70,4,4,4,444
13751.0,288,9,296.25,4,4,4,444
16998.0,149,4,295.00,4,4,4,444
15732.0,212,14,294.12,4,4,4,444
12829.0,336,11,293.00,4,4,4,444
16526.0,171,15,290.36,4,4,4,444
